In [1]:
from simpletransformers.question_answering import QuestionAnsweringModel
import os
import json
import transformers
import torch
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
print(torch.cuda.is_available(), torch.cuda.current_device())

True 0


### Training

In [2]:
# load train data into a dictionary
with open('../../data/train-v2.0.json', 'r') as f:
    train_data = json.load(f)
train_data = [item for topic in train_data['data'] for item in topic['paragraphs'] ]

len(train_data)

19035

In [3]:
train_args = {
    'output_dir': 'checkpoints/',
    'overwrite_output_dir': True,
    'learning_rate': 3e-5,
    'num_train_epochs': 2,
    'max_seq_length': 384,
    'doc_stride': 128,
    'overwrite_output_dir': True,
    'reprocess_input_data': False,
    'train_batch_size': 8,
    'gradient_accumulation_steps': 8,
}
model = QuestionAnsweringModel(model_type='distilbert', 
                               model_name='distilbert-base-uncased-distilled-squad',
                               args=train_args, 
                               use_cuda=True,
                               cuda_device=0)

In [ ]:
model.train_model(train_data=train_data, output_dir='checkpoints/', use_cuda=True, cuda_device=0)

convert squad examples to features:  70%|███████   | 91501/130319 [05:29<40:29, 15.98it/s]   Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/pallavi/anaconda3/envs/venv/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/pallavi/anaconda3/envs/venv/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pallavi/anaconda3/envs/venv/lib/python3.6/multiprocessing/pool.py", line 405, in _handle_workers
    pool._maintain_pool()
  File "/home/pallavi/anaconda3/envs/venv/lib/python3.6/multiprocessing/pool.py", line 246, in _maintain_pool
    self._repopulate_pool()
  File "/home/pallavi/anaconda3/envs/venv/lib/python3.6/multiprocessing/pool.py", line 239, in _repopulate_pool
    w.start()
  File "/home/pallavi/anaconda3/envs/venv/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/home/pallavi/anaconda3/envs/venv/lib/pytho

### Evaluation on dev data

In [ ]:
# load the trained model
model = QuestionAnsweringModel(model_type='distilbert', 
                               model_name='checkpoints/', 
                               args=train_args, 
                               use_cuda=True)

In [ ]:
# load dev data into a dictionary
with open('../../data/dev-v2.0.json', 'r') as f:
    dev_data = json.load(f)
    
dev_data = [item for topic in dev_data['data'] for item in topic['paragraphs'] ]

len(dev_data)

In [ ]:
preds, _ = model.predict(dev_data)

predictions = {pred['id']: pred['answer'][0] for pred in preds}
len(predictions)

In [ ]:
# write output to predictions file
os.makedirs('output', exist_ok=True)
with open('output/predictions.json', 'w') as f:
    json.dump(predictions, f)